In [1]:
from autumn.projects.sm_covid2.common_school.project_maker import get_school_project_parameter_set
from autumn.projects.sm_covid2.common_school.calibration import get_bcm_object
import pandas as pd
from copy import copy
from pathlib import Path

c:\Users\rrag0004\.conda\envs\summer2\lib\site-packages\summer\runner\vectorized_runner.py:363: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_strain_infection_values(
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
bcm = get_bcm_object("FRA")

In [3]:
params_info = pd.read_excel("params_info.xlsx")

In [4]:
iso3 = "FRA"
first_date_with_death = 0
sero_age_min = 0
sero_age_max = 100
analysis='main'

In [5]:
model_params = get_school_project_parameter_set(
    iso3,
    first_date_with_death,
    sero_age_min,
    sero_age_max,
    analysis
).baseline

In [6]:
def get_param_val(p_name, model_params):
    if "." not in p_name:
        return model_params[p_name]
    else:
        nested_par = model_params
        nested_keys = p_name.split(".")
        for k in nested_keys:
            nested_par = nested_par[k]        
        return nested_par

In [7]:
def make_output_params_df(params_info, model_params):
    params_df = copy(params_info)

    for index in params_df.index:
        p_name = params_df.loc[index, 'param_key']
        value = params_df.loc[index, 'value']
        if value == "LOAD":
            if p_name in bcm.priors:
                p = bcm.priors[p_name]
                bounds = p.bounds()
                displayed_value = f"Uniform ({bounds[0]}, {bounds[1]})"
            else:
                displayed_value = get_param_val(p_name, model_params)
            params_df.loc[index, 'value'] = displayed_value
        elif value in ("HEADER", "SUBHEADER"):
            params_df.loc[index, 'value'] = ""
            params_df.loc[index, 'unit'] = ""
            params_df.loc[index, 'evidence'] = ""

            header_name = params_df.loc[index, 'legible_name']
            if value == "HEADER":
                params_df.loc[index, 'legible_name'] = f"BEGINBOLD{header_name}ENDBOLD"
            else:  # "SUBHEADER"
                params_df.loc[index, 'legible_name'] = f"BEGINITAL{header_name}ENDITAL"

        # tag calibrated parameters
        if p_name in bcm.priors:
            legible_name = params_df.loc[index, 'legible_name']
            params_df.loc[index, 'legible_name'] = f"{legible_name} SUPERC"

    return params_df

In [8]:
df = make_output_params_df(params_info, model_params)

In [9]:
pd.set_option('display.max_colwidth', None)
tex_table = df.to_latex(
    columns=['legible_name', 'value', 'evidence'],
    header=['Parameter', "Value/Distribution", "Evidence"],
    index=False,
    label="param_table",
    caption="Model parameters"
)

In [10]:
# Handle citations
tex_table = tex_table.replace("\\textbackslash cite\\", r"\cite").replace("\\}", "}")

In [11]:
# Equation mode not captured
tex_table = tex_table.replace("(\$m\_S\$)", r"($m_S$)")
tex_table = tex_table.replace("(\$m\_C\$)", r"($m_C$)")


In [12]:
# Use bold and italic font types for headings and sub-headings
tex_table = tex_table.replace("BEGINBOLD", r"\textbf{").replace("ENDBOLD", r"}")
tex_table = tex_table.replace("BEGINITAL", r"\quad \textit{").replace("ENDITAL", r"}")

In [13]:
# include superscript C for calibrated parameters
tex_table = tex_table.replace("SUPERC", r"\textsuperscript{C}")

In [14]:
# Change the table font so its width matches the pagewidth 
tex_table = tex_table.replace("\\begin{tabular}", r"\resizebox{\textwidth}{!}{\begin{tabular}")
tex_table = tex_table.replace("\\end{tabular}", r"\end{tabular}}")

In [15]:
out_path = Path.home() / "Models"/ "AuTuMN_new" / "docs" / "tex" / "tex_descriptions" / "projects" / "sm_covid" / "param_table.tex"

In [16]:
with out_path.open("w") as f:
    f.write(tex_table)